In [24]:
import pandas as pd
import numpy as np
#Data analysis
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# For model buidling
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from pymongo import MongoClient

# Custom module for prediction and model evalution
from utils import predict_and_evaluate

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error

import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from missingpy import MissForest

from utils import predict_and_evaluate

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# Missing value treatment

In [ ]:
#Connecting to mongo db and loading all data

# cluster=MongoClient('mongodb://localhost:27017')
# db=cluster['medicare']
# collection = db['final_data']
# df = pd.DataFrame(list(collection.find()))

In [2]:
#Or we can load from local

df = pd.read_csv("../../../data/final_dataframe.csv")

In [3]:
df.isnull().sum()

Unnamed: 0                  0
NPI                         0
City                        0
State                       0
Speciality                  1
max_Tot_Drug_Cst            0
sum_Tot_Drug_Cst            0
avg_Tot_Drug_Cst            0
max_Total_claims            0
sum_Total_claims            0
avg_Total_claims            0
max_Tot_Day_Suply           0
sum_Tot_Day_Suply           0
avg_Tot_Day_Suply           0
max_Tot_30day_Fills         0
sum_Tot_30day_Fills         0
avg_Tot_30day_Fills         0
Total_payments         624634
FRAUD                       0
dtype: int64

In [11]:
df.Total_payments.isnull().sum()/965056

0.6472515584587838

In [3]:
#df.drop(["Unnamed: 0","State","City"],axis =1,inplace= True)
df.drop("Unnamed: 0",axis =1,inplace= True)
df = df.round(2)

In [4]:
df.FRAUD.value_counts()

0    964170
1    886   
Name: FRAUD, dtype: int64

In [4]:
df.isnull().sum()

NPI                    0     
City                   0     
State                  0     
Speciality             1     
max_Tot_Drug_Cst       0     
sum_Tot_Drug_Cst       0     
avg_Tot_Drug_Cst       0     
max_Total_claims       0     
sum_Total_claims       0     
avg_Total_claims       0     
max_Tot_Day_Suply      0     
sum_Tot_Day_Suply      0     
avg_Tot_Day_Suply      0     
max_Tot_30day_Fills    0     
sum_Tot_30day_Fills    0     
avg_Tot_30day_Fills    0     
Total_payments         624634
FRAUD                  0     
dtype: int64

In [5]:
df.head(10)

NPI       City State               Speciality  max_Tot_Drug_Cst  \
0  1003000126  bethesda   md    internal medicine        4792.85            
1  1003000423  cleveland  oh    obstetrics & gynecology  8276.44            
2  1003000720  clemmons   nc    nurse practitioner       657.01             
3  1003001785  tulsa      ok    orthopedic surgery       2141.24            
4  1003001884  flint      mi    family practice          1324.26            
5  1003002049  fremont    ca    endocrinology            74927.70           
6  1003002312  quincy     ma    rheumatology             331200.50          
7  1003002379  portland   or    internal medicine        7866.33            
8  1003002817  doral      fl    family practice          163725.45          
9  1003002890  brighton   co    orthopedic surgery       723.36             

   sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  sum_Total_claims  \
0  5979.07           747.38            30                124                
1  15389.69          2564.95           39                122                
2  3052.84           254.40            51                230                
3  7812.28           781.23            126               546                
4  2449.09           349.87            31                118                
5  443067.28         13845.85          130               984                
6  752350.50         11063.98          310               2693               
7  10079.20          1259.90           41                152                
8  1063755.80        5089.74           871               18489              
9  1011.62           252.90            27                82                 

   avg_Total_claims  max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
0  15.50             937                3721               465.12              
1  20.33             2412               5217               869.50              
2  19.17             337                2166               180.50              
3  54.60             2610               8250               825.00              
4  16.86             1560               4979               711.29              
5  30.75             9969               75850              2370.31             
6  39.60             11852              121124             1781.24             
7  19.00             1448               4328               541.00              
8  88.46             76881              982587             4701.37             
9  20.50             765                2155               538.75              

   max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  \
0  32.4                 138.2                17.27                 
1  82.0                 213.7                35.62                 
2  51.0                 230.0                19.17                 
3  126.0                569.0                56.90                 
4  53.0                 199.0                28.43                 
5  333.9                2534.6               79.21                 
6  396.0                4126.1               60.68                 
7  56.0                 181.3                22.66                 
8  2563.5               34647.8              165.78                
9  30.3                 96.8                 24.20                 

   Total_payments  FRAUD  
0  20.48           0      
1  171.29          0      
2 NaN              0      
3  969.07          0      
4 NaN              0      
5  20.61           0      
6  5422.67         0      
7 NaN              0      
8 NaN              0      
9  227.34          0

In [6]:
df_sample = df[df["Total_payments"].notnull()].sample(n=50)
df_sample.head()

NPI             City State         Speciality  \
560934  1346230521  boston           ma    endocrinology       
704195  1932180817  west bloomfield  mi    internal medicine   
26489   1104993500  scottsdale       az    psychiatry          
662772  1760474605  norman           ok    internal medicine   
539079  1255363651  orlando          fl    internal medicine   

        max_Tot_Drug_Cst  sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  \
560934  288965.84         515277.06         36805.50           
704195  25754.68          231984.30         2252.27            
26489   84992.19          99661.66          4745.79            
662772  161963.38         1067682.00        4766.44            
539079  9011.55           12805.41          800.34             

        max_Total_claims  sum_Total_claims  avg_Total_claims  \
560934  197               706               50.43              
704195  224               4186              40.64              
26489   53                502               23.90              
662772  962               16621             74.20              
539079  56                338               21.12              

        max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
560934  15273              42544              3038.86             
704195  19900              275535             2675.10             
26489   1495               18873              898.71              
662772  46361              776268             3465.48             
539079  930                5449               340.56              

        max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  \
560934  511.6                1437.1               102.65                
704195  663.5                9576.4               92.97                 
26489   53.0                 648.3                30.87                 
662772  1748.0               28326.9              126.46                
539079  56.0                 343.1                21.44                 

        Total_payments  FRAUD  
560934  8716.30         0      
704195  164.64          0      
26489   383.64          0      
662772  135.78          0      
539079  53.68           0

In [7]:
df_1 = df.copy()

### Encoding

In [8]:
inds = df_sample.index.to_list()
for i in inds:
    df_1.at[i,"Total_payments"] = np.nan

In [9]:
df_1.iloc[inds]

NPI              City State  \
560934  1346230521  boston            ma     
704195  1932180817  west bloomfield   mi     
26489   1104993500  scottsdale        az     
662772  1760474605  norman            ok     
539079  1255363651  orlando           fl     
147018  1609256700  hialeah           fl     
914541  1801992029  thurmont          md     
725509  1043347222  point pleasant    wv     
413092  1710220389  charlotte         nc     
189525  1770926149  jacksonville      fl     
232422  1952669152  owensboro         ky     
628243  1619972015  nashua            nh     
646299  1699710004  johnson city      tn     
318441  1336132844  orange city       fl     
910805  1790735215  dubuque           ia     
411684  1700881489  artesia           nm     
434454  1790983450  greenvale         ny     
767469  1215225412  norwalk           oh     
255143  1073596813  sumter            sc     
713736  1972507341  harleysville      pa     
90620   1376510958  jasper            in     
658940  1740548676  brenham           tx     
717607  1013112911  south plainfield  nj     
768950  1215981766  saint louis       mo     
838270  1497954200  greenville        nc     
190179  1780145755  wynnewood         pa     
54706   1225226384  mission viejo     ca     
921412  1831504018  indianapolis      in     
449589  1861414286  greenville        nc     
754843  1164487468  havertown         pa     
944870  1932209855  knoxville         tn     
855263  1568825222  west hartford     ct     
754975  1164502340  bay city          mi     
679675  1831176775  swansea           il     
568634  1376545145  hattiesburg       ms     
793317  1316914609  rochester         ny     
809769  1386630911  jackson           tn     
39308   1164462321  mobile            al     
870841  1639164825  st. petersburg    fl     
922477  1841266202  bradenton         fl     
16128   1063726925  haddon heights    nj     
699170  1912089251  macon             ga     
772074  1235172164  nashville         tn     
272559  1144457490  pittsburgh        pa     
676374  1811962723  san antonio       tx     
672012  1801028717  laredo            tx     
197111  1811043540  peachtree city    ga     
170356  1699899617  rainelle          wv     
321731  1346397239  sheffield         al     
218590  1902023476  baldwin park      ca     

                                                            Speciality  \
560934  endocrinology                                                    
704195  internal medicine                                                
26489   psychiatry                                                       
662772  internal medicine                                                
539079  internal medicine                                                
147018  dentist                                                          
914541  family practice                                                  
725509  family practice                                                  
413092  hospitalist                                                      
189525  emergency medicine                                               
232422  sports medicine                                                  
628243  ophthalmology                                                    
646299  nurse practitioner                                               
318441  hematology-oncology                                              
910805  orthopedic surgery                                               
411684  obstetrics & gynecology                                          
434454  pulmonary disease                                                
767469  family practice                                                  
255143  hematology-oncology                                              
713736  family practice                                                  
90620   general surgery                                                  
658940  family practice                 

In [25]:
imputer = MissForest()

In [11]:
#df_imputed = imputer.fit_transform(df_1.drop(["NPI","State","Speciality","FRAUD"],axis = 1))
df_imputed = imputer.fit_transform(df_1.drop(["NPI","City","State","Speciality","FRAUD"],axis = 1))

C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 0


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 2


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 3


In [12]:
df_2 = df.copy()

In [13]:
df_2.head(2)

NPI       City State               Speciality  max_Tot_Drug_Cst  \
0  1003000126  bethesda   md    internal medicine        4792.85            
1  1003000423  cleveland  oh    obstetrics & gynecology  8276.44            

   sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  sum_Total_claims  \
0  5979.07           747.38            30                124                
1  15389.69          2564.95           39                122                

   avg_Total_claims  max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
0  15.50             937                3721               465.12              
1  20.33             2412               5217               869.50              

   max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  \
0  32.4                 138.2                17.27                 
1  82.0                 213.7                35.62                 

   Total_payments  FRAUD  
0  20.48           0      
1  171.29          0

In [14]:
df_2["Payments_all"] = df_imputed[:,-1]
df_2.iloc[inds]

#Poor performance of missforest algorithm
#With just avg columns
#with all numeric columns

#On encoding of speciality column gaved all payments as 0,1
#0 959887
#1 5169
#total 965056
#Hence encoding is not a good option

NPI              City State  \
560934  1346230521  boston            ma     
704195  1932180817  west bloomfield   mi     
26489   1104993500  scottsdale        az     
662772  1760474605  norman            ok     
539079  1255363651  orlando           fl     
147018  1609256700  hialeah           fl     
914541  1801992029  thurmont          md     
725509  1043347222  point pleasant    wv     
413092  1710220389  charlotte         nc     
189525  1770926149  jacksonville      fl     
232422  1952669152  owensboro         ky     
628243  1619972015  nashua            nh     
646299  1699710004  johnson city      tn     
318441  1336132844  orange city       fl     
910805  1790735215  dubuque           ia     
411684  1700881489  artesia           nm     
434454  1790983450  greenvale         ny     
767469  1215225412  norwalk           oh     
255143  1073596813  sumter            sc     
713736  1972507341  harleysville      pa     
90620   1376510958  jasper            in     
658940  1740548676  brenham           tx     
717607  1013112911  south plainfield  nj     
768950  1215981766  saint louis       mo     
838270  1497954200  greenville        nc     
190179  1780145755  wynnewood         pa     
54706   1225226384  mission viejo     ca     
921412  1831504018  indianapolis      in     
449589  1861414286  greenville        nc     
754843  1164487468  havertown         pa     
944870  1932209855  knoxville         tn     
855263  1568825222  west hartford     ct     
754975  1164502340  bay city          mi     
679675  1831176775  swansea           il     
568634  1376545145  hattiesburg       ms     
793317  1316914609  rochester         ny     
809769  1386630911  jackson           tn     
39308   1164462321  mobile            al     
870841  1639164825  st. petersburg    fl     
922477  1841266202  bradenton         fl     
16128   1063726925  haddon heights    nj     
699170  1912089251  macon             ga     
772074  1235172164  nashville         tn     
272559  1144457490  pittsburgh        pa     
676374  1811962723  san antonio       tx     
672012  1801028717  laredo            tx     
197111  1811043540  peachtree city    ga     
170356  1699899617  rainelle          wv     
321731  1346397239  sheffield         al     
218590  1902023476  baldwin park      ca     

                                                            Speciality  \
560934  endocrinology                                                    
704195  internal medicine                                                
26489   psychiatry                                                       
662772  internal medicine                                                
539079  internal medicine                                                
147018  dentist                                                          
914541  family practice                                                  
725509  family practice                                                  
413092  hospitalist                                                      
189525  emergency medicine                                               
232422  sports medicine                                                  
628243  ophthalmology                                                    
646299  nurse practitioner                                               
318441  hematology-oncology                                              
910805  orthopedic surgery                                               
411684  obstetrics & gynecology                                          
434454  pulmonary disease                                                
767469  family practice                                                  
255143  hematology-oncology                                              
713736  family practice                                                  
90620   general surgery                                                  
658940  family practice                 

In [15]:
df_imputed_2 = imputer.fit_transform(df_1.loc[:,["avg_Tot_Drug_Cst","avg_Total_claims","avg_Tot_Day_Suply","avg_Tot_30day_Fills","Total_payments"]])

C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 0


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 2


C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\Shailesh\Desktop\project\myenv\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 3


In [16]:
df_2["Payments_avg"] = df_imputed_2[:,-1]
df_2.iloc[inds]

NPI              City State  \
560934  1346230521  boston            ma     
704195  1932180817  west bloomfield   mi     
26489   1104993500  scottsdale        az     
662772  1760474605  norman            ok     
539079  1255363651  orlando           fl     
147018  1609256700  hialeah           fl     
914541  1801992029  thurmont          md     
725509  1043347222  point pleasant    wv     
413092  1710220389  charlotte         nc     
189525  1770926149  jacksonville      fl     
232422  1952669152  owensboro         ky     
628243  1619972015  nashua            nh     
646299  1699710004  johnson city      tn     
318441  1336132844  orange city       fl     
910805  1790735215  dubuque           ia     
411684  1700881489  artesia           nm     
434454  1790983450  greenvale         ny     
767469  1215225412  norwalk           oh     
255143  1073596813  sumter            sc     
713736  1972507341  harleysville      pa     
90620   1376510958  jasper            in     
658940  1740548676  brenham           tx     
717607  1013112911  south plainfield  nj     
768950  1215981766  saint louis       mo     
838270  1497954200  greenville        nc     
190179  1780145755  wynnewood         pa     
54706   1225226384  mission viejo     ca     
921412  1831504018  indianapolis      in     
449589  1861414286  greenville        nc     
754843  1164487468  havertown         pa     
944870  1932209855  knoxville         tn     
855263  1568825222  west hartford     ct     
754975  1164502340  bay city          mi     
679675  1831176775  swansea           il     
568634  1376545145  hattiesburg       ms     
793317  1316914609  rochester         ny     
809769  1386630911  jackson           tn     
39308   1164462321  mobile            al     
870841  1639164825  st. petersburg    fl     
922477  1841266202  bradenton         fl     
16128   1063726925  haddon heights    nj     
699170  1912089251  macon             ga     
772074  1235172164  nashville         tn     
272559  1144457490  pittsburgh        pa     
676374  1811962723  san antonio       tx     
672012  1801028717  laredo            tx     
197111  1811043540  peachtree city    ga     
170356  1699899617  rainelle          wv     
321731  1346397239  sheffield         al     
218590  1902023476  baldwin park      ca     

                                                            Speciality  \
560934  endocrinology                                                    
704195  internal medicine                                                
26489   psychiatry                                                       
662772  internal medicine                                                
539079  internal medicine                                                
147018  dentist                                                          
914541  family practice                                                  
725509  family practice                                                  
413092  hospitalist                                                      
189525  emergency medicine                                               
232422  sports medicine                                                  
628243  ophthalmology                                                    
646299  nurse practitioner                                               
318441  hematology-oncology                                              
910805  orthopedic surgery                                               
411684  obstetrics & gynecology                                          
434454  pulmonary disease                                                
767469  family practice                                                  
255143  hematology-oncology                                              
713736  family practice                                                  
90620   general surgery                                                  
658940  family practice                 

In [17]:
#df_2["payments"] = df_2["Total_payments"]

In [18]:
df_2.iloc[inds]

NPI              City State  \
560934  1346230521  boston            ma     
704195  1932180817  west bloomfield   mi     
26489   1104993500  scottsdale        az     
662772  1760474605  norman            ok     
539079  1255363651  orlando           fl     
147018  1609256700  hialeah           fl     
914541  1801992029  thurmont          md     
725509  1043347222  point pleasant    wv     
413092  1710220389  charlotte         nc     
189525  1770926149  jacksonville      fl     
232422  1952669152  owensboro         ky     
628243  1619972015  nashua            nh     
646299  1699710004  johnson city      tn     
318441  1336132844  orange city       fl     
910805  1790735215  dubuque           ia     
411684  1700881489  artesia           nm     
434454  1790983450  greenvale         ny     
767469  1215225412  norwalk           oh     
255143  1073596813  sumter            sc     
713736  1972507341  harleysville      pa     
90620   1376510958  jasper            in     
658940  1740548676  brenham           tx     
717607  1013112911  south plainfield  nj     
768950  1215981766  saint louis       mo     
838270  1497954200  greenville        nc     
190179  1780145755  wynnewood         pa     
54706   1225226384  mission viejo     ca     
921412  1831504018  indianapolis      in     
449589  1861414286  greenville        nc     
754843  1164487468  havertown         pa     
944870  1932209855  knoxville         tn     
855263  1568825222  west hartford     ct     
754975  1164502340  bay city          mi     
679675  1831176775  swansea           il     
568634  1376545145  hattiesburg       ms     
793317  1316914609  rochester         ny     
809769  1386630911  jackson           tn     
39308   1164462321  mobile            al     
870841  1639164825  st. petersburg    fl     
922477  1841266202  bradenton         fl     
16128   1063726925  haddon heights    nj     
699170  1912089251  macon             ga     
772074  1235172164  nashville         tn     
272559  1144457490  pittsburgh        pa     
676374  1811962723  san antonio       tx     
672012  1801028717  laredo            tx     
197111  1811043540  peachtree city    ga     
170356  1699899617  rainelle          wv     
321731  1346397239  sheffield         al     
218590  1902023476  baldwin park      ca     

                                                            Speciality  \
560934  endocrinology                                                    
704195  internal medicine                                                
26489   psychiatry                                                       
662772  internal medicine                                                
539079  internal medicine                                                
147018  dentist                                                          
914541  family practice                                                  
725509  family practice                                                  
413092  hospitalist                                                      
189525  emergency medicine                                               
232422  sports medicine                                                  
628243  ophthalmology                                                    
646299  nurse practitioner                                               
318441  hematology-oncology                                              
910805  orthopedic surgery                                               
411684  obstetrics & gynecology                                          
434454  pulmonary disease                                                
767469  family practice                                                  
255143  hematology-oncology                                              
713736  family practice                                                  
90620   general surgery                                                  
658940  family practice                 

In [19]:
print("MSE with all numeric features : ",mean_squared_error(df_2.iloc[inds]["Total_payments"].to_list(), df_2.iloc[inds]["Payments_all"].to_list()))
print("RMSE with all numeric features : ",mean_squared_error(df_2.iloc[inds]["Total_payments"].to_list(), df_2.iloc[inds]["Payments_all"],squared = False))
print("MSE with avg numeric features : ",mean_squared_error(df_2.iloc[inds]["Total_payments"].to_list(), df_2.iloc[inds]["Payments_avg"].to_list()))
print("RMSE with avg numeric features : ",mean_squared_error(df_2.iloc[inds]["Total_payments"].to_list(), df_2.iloc[inds]["Payments_avg"].to_list(),squared = False))

MSE with all numeric features :  23568038.22044275
RMSE with all numeric features :  4854.692391948511
MSE with avg numeric features :  21743392.210048605
RMSE with avg numeric features :  4662.981043286431


In [20]:
df_2.head()

NPI       City State               Speciality  max_Tot_Drug_Cst  \
0  1003000126  bethesda   md    internal medicine        4792.85            
1  1003000423  cleveland  oh    obstetrics & gynecology  8276.44            
2  1003000720  clemmons   nc    nurse practitioner       657.01             
3  1003001785  tulsa      ok    orthopedic surgery       2141.24            
4  1003001884  flint      mi    family practice          1324.26            

   sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  sum_Total_claims  \
0  5979.07           747.38            30                124                
1  15389.69          2564.95           39                122                
2  3052.84           254.40            51                230                
3  7812.28           781.23            126               546                
4  2449.09           349.87            31                118                

   avg_Total_claims  max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
0  15.50             937                3721               465.12              
1  20.33             2412               5217               869.50              
2  19.17             337                2166               180.50              
3  54.60             2610               8250               825.00              
4  16.86             1560               4979               711.29              

   max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  \
0  32.4                 138.2                17.27                 
1  82.0                 213.7                35.62                 
2  51.0                 230.0                19.17                 
3  126.0                569.0                56.90                 
4  53.0                 199.0                28.43                 

   Total_payments  FRAUD  Payments_all  Payments_avg  
0  20.48           0      20.48000      20.48000      
1  171.29          0      171.29000     171.29000     
2 NaN              0      4719.73420    281.08064     
3  969.07          0      969.07000     969.07000     
4 NaN              0      157.69675     322.85420

In [21]:
df_spec = df_2.loc[:,["Speciality","Total_payments"]]

In [22]:
df_gr = df_spec.groupby("Speciality").Total_payments.agg(["min","max","mean","std","var","count"])

### Attaching payments column to hyp data set

In [24]:
df_hyp = pd.read_csv("../data/hyp_0.1_data.csv")

In [26]:
df_hyp.drop("Total_payments",axis =1,inplace = True)
#df_2.loc[inds,"Payments_all"]= df_2.iloc[inds]["Total_payments"].values
df_2 = df_2.round(2)
df_hyp["Total_payments"] = df_2["Payments_all"]

In [27]:
df_hyp.head()

Unnamed: 0         NPI State               Speciality  max_Tot_Drug_Cst  \
0  0           1003000126  md    internal medicine        4792.85            
1  1           1003000423  oh    obstetrics & gynecology  8276.44            
2  2           1003000720  nc    nurse practitioner       657.01             
3  3           1003001785  ok    orthopedic surgery       2141.24            
4  4           1003001884  mi    family practice          1324.26            

   sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  sum_Total_claims  \
0  5979.07           747.38370         30                124                
1  15389.69          2564.94820        39                122                
2  3052.84           254.40334         51                230                
3  7812.28           781.22800         126               546                
4  2449.09           349.87000         31                118                

   avg_Total_claims  max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
0  15.500000         937                3721               465.1250            
1  20.333333         2412               5217               869.5000            
2  19.166667         337                2166               180.5000            
3  54.600000         2610               8250               825.0000            
4  16.857143         1560               4979               711.2857            

   max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  FRAUD  \
0  32.4                 138.2                17.275000            0       
1  82.0                 213.7                35.616665            0       
2  51.0                 230.0                19.166666            0       
3  126.0                569.0                56.900000            0       
4  53.0                 199.0                28.428572            0       

   claim_max-mean  supply_max-mean  drug_max-mean  Methocarbamol_Tot_Drug_Cst  \
0  14.500000       471.8750         4045.46630     0.0                          
1  18.666667       1542.5000        5711.49180     0.0                          
2  31.833333       156.5000         402.60666      0.0                          
3  71.400000       1785.0000        1360.01200     0.0                          
4  14.142857       848.7143         974.39000      0.0                          

   Atorvastatin Calcium_Tot_Drug_Cst  Atorvastatin Calcium_Tot_Clms  \
0  0.0                                0.0                             
1  0.0                                0.0                             
2  0.0                                0.0                             
3  0.0                                0.0                             
4  0.0                                0.0                             

   Atorvastatin Calcium_Tot_Day_Suply  Gabapentin_Tot_Drug_Cst  \
0  0.0                                 0.0                       
1  0.0                                 0.0                       
2  0.0                                 0.0                       
3  0.0                                 0.0                       
4  0.0                                 0.0                       

   Gabapentin_Tot_Clms  Gabapentin_Tot_Day_Suply  Abatacept_Tot_Drug_Cst  \
0  0.0                  0.0                       0.0                      
1  0.0                  0.0                       0.0                      
2  0.0                  0.0                       0.0                      
3  0.0                  0.0                       0.0                      
4  0.0                  0.0                       0.0                      

   Abatacept_Tot_Clms  Abatacept_Tot_Day_Suply  \
0  0.0                 0.0                       
1  0.0                 0.0                       
2  0.0                 0.0                       
3  0.0                 0.0                       
4  0.0                 0.0                       

   Risperidone Microspheres_Tot_Drug_Cst  \
0  0.0                    

In [28]:
df_hyp["Total_payments"].isnull().sum()

0

In [ ]:
df_2.head()

In [29]:
# df_hyp.to_csv("../data/hyp_pay_fill_0.1.csv")

# Hypothesis test

## Loading data

In [ ]:
#orignal data from cms.gov.in
df = pd.read_csv("MUP_DPR_RY21_P04_V10_DY20_NPIBN_0.csv")

In [ ]:
df.head()

In [ ]:
# #Connecting to mongo db and loading all data

# cluster=MongoClient('mongodb://localhost:27017')
# db=cluster['medicare']
# collection = db['final_data']
# df_final = pd.DataFrame(list(collection.find()))

In [35]:
#Or we can load from loca

# df_final = pd.read_csv("../data/final_dataframe.csv")

In [36]:
# df_final.drop("Unnamed: 0",axis = 1,inplace = True)

In [ ]:
#copying missing value filled data to new variable

df_final = df_hyp.copy()

In [37]:
df_final.head(1)

NPI      City State         Speciality  max_Tot_Drug_Cst  \
0  1003000126  bethesda    md  internal medicine           4792.85   

   sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  sum_Total_claims  \
0           5979.07          747.3837                30               124   

   avg_Total_claims  max_Tot_Day_Suply  sum_Tot_Day_Suply  avg_Tot_Day_Suply  \
0              15.5                937               3721            465.125   

   max_Tot_30day_Fills  sum_Tot_30day_Fills  avg_Tot_30day_Fills  \
0                 32.4                138.2               17.275   

   Total_payments  FRAUD  
0           20.48      0

In [29]:
df_final1 = df_final.loc[:,["NPI","FRAUD"]]

In [30]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965056 entries, 0 to 965055
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   NPI     965056 non-null  int64
 1   FRAUD   965056 non-null  int64
dtypes: int64(2)
memory usage: 14.7 MB


In [39]:
df_final1.FRAUD.value_counts()

0    964170
1       886
Name: FRAUD, dtype: int64

## Pandas

In [ ]:
df.info()

In [ ]:
#Selecting only the required columns from original dataset

df = df.loc[:,["Prscrbr_NPI","Gnrc_Name","Tot_Clms","Tot_30day_Fills","Tot_Day_Suply","Tot_Drug_Cst","Prscrbr_Type"]]

In [ ]:
df.info()

In [ ]:
#renaming

df.columns = ['NPI', 'Drug_Name', 'Tot_Clms', 'Tot_30day_Fills','Tot_Day_Suply','Tot_Drug_Cst','Speciality']

In [ ]:
df.info()

In [ ]:
df.head(1)

In [ ]:
partD_drug_train = df.merge(df_final1,on='NPI',how='inner')

In [ ]:
partD_drug_train.info()

In [ ]:
partD_drug_train.head()

In [ ]:
#saving dataset for future use and quick reloading , as size of orignal dataset is huge

partD_drug_train.to_csv("Drug_20")

## Loading data : csv data : dataframe with original drug data and fraud

In [ ]:
partD_drug = pd.read_csv("../data/Drug_20.csv")

In [5]:
partD_drug.head(1)

Unnamed: 0         NPI       Drug_Name  Tot_Clms  Tot_30day_Fills  \
0           0  1003000126  Amiodarone Hcl        13             13.8   

   Tot_Day_Suply  Tot_Drug_Cst         Speciality  FRAUD  
0            399        135.38  Internal Medicine      0

In [6]:
partD_drug.drop("Unnamed: 0",axis = 1,inplace = True)

In [7]:
partD_drug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25048504 entries, 0 to 25048503
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   NPI              int64  
 1   Drug_Name        object 
 2   Tot_Clms         int64  
 3   Tot_30day_Fills  float64
 4   Tot_Day_Suply    int64  
 5   Tot_Drug_Cst     float64
 6   Speciality       object 
 7   FRAUD            int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.5+ GB


## Grouping data 

In [8]:
partD_drug_Group = partD_drug.groupby(["Drug_Name","FRAUD"])

In [9]:
partD_drug_Group.first()

NPI  Tot_Clms  Tot_30day_Fills  \
Drug_Name                      FRAUD                                          
0.9 % Sodium Chloride          0      1003008533        16             16.0   
                               1      1285627828        20             20.0   
Aa 4.25%/Calcium/Lytes/Dex 10% 0      1033137518        11             11.0   
Aa 5 %/Calcium/Lytes/Dext 20 % 0      1356304224        33             33.0   
Aa 5%/D15w/Electrolytes        0      1104087469        23             23.0   
...                                          ...       ...              ...   
Zolpidem Tartrate              0      1003000522        22             46.0   
                               1      1003806431        41             45.0   
Zonisamide                     0      1003000902        12             12.0   
                               1      1629042304        11             11.0   
Zoster Vaccine Live/Pf         0      1255609913        11             11.0   

                                      Tot_Day_Suply  Tot_Drug_Cst  \
Drug_Name                      FRAUD                                
0.9 % Sodium Chloride          0                401        120.56   
                               1                315        333.86   
Aa 4.25%/Calcium/Lytes/Dex 10% 0                 76       5136.48   
Aa 5 %/Calcium/Lytes/Dext 20 % 0                161      12101.10   
Aa 5%/D15w/Electrolytes        0                161       7782.00   
...                                             ...           ...   
Zolpidem Tartrate              0               1380        273.87   
                               1               1310         97.95   
Zonisamide                     0                360        654.89   
                               1                330        264.45   
Zoster Vaccine Live/Pf         0                 11       2464.22   

                                               Speciality  
Drug_Name                      FRAUD                       
0.9 % Sodium Chloride          0                Neurology  
                               1          Family Practice  
Aa 4.25%/Calcium/Lytes/Dex 10% 0      Hematology-Oncology  
Aa 5 %/Calcium/Lytes/Dext 20 % 0          General Surgery  
Aa 5%/D15w/Electrolytes        0        Internal Medicine  
...                                                   ...  
Zolpidem Tartrate              0          Family Practice  
                               1        Internal Medicine  
Zonisamide                     0          Family Practice  
                               1        Internal Medicine  
Zoster Vaccine Live/Pf         0       Nurse Practitioner  

[2409 rows x 6 columns]

## Pre process Hypothesis test

In [10]:
# storing unique drug names in set
#is instance will return false if drug name is not a string && set datatype - for no duplicacy
drugs = set([ drugx for drugx in partD_drug['Drug_Name'].values if isinstance(drugx, str)])
print(len(drugs))

1722


In [11]:
drug_keys = partD_drug_Group.groups.keys()
print(len(drug_keys))

2409


In [12]:
list(drug_keys)[0:10]

[('0.9 % Sodium Chloride', 0),
 ('0.9 % Sodium Chloride', 1),
 ('Aa 4.25%/Calcium/Lytes/Dex 10%', 0),
 ('Aa 5 %/Calcium/Lytes/Dext 20 %', 0),
 ('Aa 5%/D15w/Electrolytes', 0),
 ('Abacavir Sulfate', 0),
 ('Abacavir Sulfate/Lamivudine', 0),
 ('Abacavir/Dolutegravir/Lamivudi', 0),
 ('Abacavir/Dolutegravir/Lamivudi', 1),
 ('Abacavir/Lamivudine/Zidovudine', 0)]

In [13]:
#Getting list of drug names which where bought by both fraud and non fraud, to test hypothesis

drug_with_isfraud = [drugx for drugx in drugs if ((drugx,0.0) in drug_keys ) & ( (drugx,1.0) in drug_keys)]
drug_with_isfraud[0:10]

['Olmesartan/Hydrochlorothiazide',
 'Penicillin V Potassium',
 'Flunisolide',
 'Methotrexate Sodium',
 'Cefadroxil',
 'Tamsulosin Hcl',
 'Epinephrine',
 'Amitriptyline Hcl',
 'Doxepin Hcl',
 'Prasugrel Hcl']

In [14]:
#687 unique drug names with both fraud and non fraud group
len(drug_with_isfraud)

687

## Hypothesis test

In [15]:
from scipy.stats import ttest_ind

In [16]:
partD_drug_Group.first()

NPI  Tot_Clms  Tot_30day_Fills  \
Drug_Name                      FRAUD                                          
0.9 % Sodium Chloride          0      1003008533        16             16.0   
                               1      1285627828        20             20.0   
Aa 4.25%/Calcium/Lytes/Dex 10% 0      1033137518        11             11.0   
Aa 5 %/Calcium/Lytes/Dext 20 % 0      1356304224        33             33.0   
Aa 5%/D15w/Electrolytes        0      1104087469        23             23.0   
...                                          ...       ...              ...   
Zolpidem Tartrate              0      1003000522        22             46.0   
                               1      1003806431        41             45.0   
Zonisamide                     0      1003000902        12             12.0   
                               1      1629042304        11             11.0   
Zoster Vaccine Live/Pf         0      1255609913        11             11.0   

                                      Tot_Day_Suply  Tot_Drug_Cst  \
Drug_Name                      FRAUD                                
0.9 % Sodium Chloride          0                401        120.56   
                               1                315        333.86   
Aa 4.25%/Calcium/Lytes/Dex 10% 0                 76       5136.48   
Aa 5 %/Calcium/Lytes/Dext 20 % 0                161      12101.10   
Aa 5%/D15w/Electrolytes        0                161       7782.00   
...                                             ...           ...   
Zolpidem Tartrate              0               1380        273.87   
                               1               1310         97.95   
Zonisamide                     0                360        654.89   
                               1                330        264.45   
Zoster Vaccine Live/Pf         0                 11       2464.22   

                                               Speciality  
Drug_Name                      FRAUD                       
0.9 % Sodium Chloride          0                Neurology  
                               1          Family Practice  
Aa 4.25%/Calcium/Lytes/Dex 10% 0      Hematology-Oncology  
Aa 5 %/Calcium/Lytes/Dext 20 % 0          General Surgery  
Aa 5%/D15w/Electrolytes        0        Internal Medicine  
...                                                   ...  
Zolpidem Tartrate              0          Family Practice  
                               1        Internal Medicine  
Zonisamide                     0          Family Practice  
                               1        Internal Medicine  
Zoster Vaccine Live/Pf         0       Nurse Practitioner  

[2409 rows x 6 columns]

### Hypothesis
### Null hypothesis : Values in all feature columns for Fraud 0 and 1 category are identical and there is no difference 
### Alternate hypothesis : They is a difference in pattern and not identical

In [17]:
partD_drug.columns

Index(['NPI', 'Drug_Name', 'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply',
       'Tot_Drug_Cst', 'Speciality', 'FRAUD'],
      dtype='object')

In [18]:
#Cols of interest
cols = ['Tot_Drug_Cst','Tot_Clms','Tot_Day_Suply']

In [19]:
#Empty dict for 
re_drug_tt = dict()

In [20]:
for drugx in drug_with_isfraud:
    for colx in cols:
        fraud_0 = partD_drug_Group.get_group((drugx,0.0))[colx].values
        fraud_1 = partD_drug_Group.get_group((drugx,1.0))[colx].values
        #print(len(fraud_0), len(fraud_1))
        
        #If column has values more than 2 then do tt test on those and add result to dictionary with drug name
        if (len(fraud_0)>2) & (len(fraud_1)>2) :
            tt = ttest_ind(fraud_0, fraud_1)
            re_drug_tt[(drugx, colx)] = tt

In [21]:
#Key is a set of Drug name and column
#Value is a set of Stat and pvalue

re_drug_tt.get(('Atorvastatin Calcium','Tot_Drug_Cst'))

Ttest_indResult(statistic=-2.860861957291366, pvalue=0.004225177525825225)

In [22]:
#Setting Probilities
#If p value is less than alpha = 0.05 then rejecting null hypothesis and adding that column in to list of prob_005

Prob_005 = [(key, pvalue) for (key, (stat, pvalue)) in re_drug_tt.items() if pvalue <=0.01]  
print(len(Prob_005))

102


In [32]:
#Check random key : drug name and column name
#at list index 100, randomly
index = 100
drug_name = Prob_005[index][0][0]
column_name = Prob_005[index][0][1]
print(drug_name,column_name)

Tramadol Hcl Tot_Clms


## Getting features from dataframe with grouped data of drug cost,supply,daily supply

In [40]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965056 entries, 0 to 965055
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   NPI                  965056 non-null  int64  
 1   City                 965056 non-null  object 
 2   State                965056 non-null  object 
 3   Speciality           965055 non-null  object 
 4   max_Tot_Drug_Cst     965056 non-null  float64
 5   sum_Tot_Drug_Cst     965056 non-null  float64
 6   avg_Tot_Drug_Cst     965056 non-null  float64
 7   max_Total_claims     965056 non-null  int64  
 8   sum_Total_claims     965056 non-null  int64  
 9   avg_Total_claims     965056 non-null  float64
 10  max_Tot_Day_Suply    965056 non-null  int64  
 11  sum_Tot_Day_Suply    965056 non-null  int64  
 12  avg_Tot_Day_Suply    965056 non-null  float64
 13  max_Tot_30day_Fills  965056 non-null  float64
 14  sum_Tot_30day_Fills  965056 non-null  float64
 15  avg_Tot_30day_Fil

In [41]:
FeaturesAll_pd = df_final.drop("City",axis = 1)

In [42]:
type(FeaturesAll_pd)

pandas.core.frame.DataFrame

In [43]:
FeaturesAll_pd['claim_max-mean'] = FeaturesAll_pd['max_Total_claims'] - FeaturesAll_pd['avg_Total_claims']

FeaturesAll_pd['supply_max-mean'] = FeaturesAll_pd['max_Tot_Day_Suply'] - FeaturesAll_pd['avg_Tot_Day_Suply']

FeaturesAll_pd['drug_max-mean'] = FeaturesAll_pd['max_Tot_Drug_Cst'] - FeaturesAll_pd['avg_Tot_Drug_Cst']

In [44]:
FeaturesAll_pd.tail(3)

NPI State        Speciality  max_Tot_Drug_Cst  \
965053  1972655934    va  medical oncology          18080.71   
965054  1972662203    ms           dentist             72.16   
965055  1972669075    ok           dentist            100.68   

        sum_Tot_Drug_Cst  avg_Tot_Drug_Cst  max_Total_claims  \
965053          24226.07         3460.8672                87   
965054            118.12           59.0600                13   
965055            100.68          100.6800                34   

        sum_Total_claims  avg_Total_claims  max_Tot_Day_Suply  \
965053               263         37.571429               6262   
965054                24         12.000000                 83   
965055                34         34.000000                165   

        sum_Tot_Day_Suply  avg_Tot_Day_Suply  max_Tot_30day_Fills  \
965053              17353             2479.0                208.7   
965054                115               57.5                 13.0   
965055                165              165.0                 34.0   

        sum_Tot_30day_Fills  avg_Tot_30day_Fills  Total_payments  FRAUD  \
965053                591.7             84.52857             NaN      0   
965054                 24.0             12.00000             NaN      0   
965055                 34.0             34.00000           13.79      0   

        claim_max-mean  supply_max-mean  drug_max-mean  
965053       49.428571           3783.0     14619.8428  
965054        1.000000             25.5        13.1000  
965055        0.000000              0.0         0.0000

## Adding columns to main dataframe

In [45]:

Feature_DrugWeighted = []
new_col_all =[]

for i, p005x in enumerate(Prob_005):
    #drug name
    drug_name = p005x[0][0]
    #Column name
    cat_name = p005x[0][1] 
    
    #Adding two strings and appending it to list
    new_col = drug_name+'_'+cat_name
    new_col_all.append(new_col)
    
    #Getting columns for fraud and non fraud
    drug_0 = partD_drug_Group.get_group((drug_name,0.0))[['NPI', cat_name]]
    drug_1 = partD_drug_Group.get_group((drug_name,1.0))[['NPI', cat_name]]
    
    #Row wise
    drug_01 = pd.concat([drug_0, drug_1])
    #print("drug_01 : ",drug_01)
    
    #Chances of Single NPI making double claims for similar medicines
    drug_02 = drug_01.groupby("NPI").sum()
    drug_02["NPI"] = drug_02.index
    drug_02.index.names = ["index"]
    
    
    #Renaming column name to new column name
    drug_02.rename(columns={cat_name: new_col}, inplace=True)
    
    #Appending dataframe to list
    Feature_DrugWeighted.append(drug_02)

In [46]:
#List with all required drug names and there values wrt NPI
Feature_DrugWeighted[0]

Methocarbamol_Tot_Drug_Cst         NPI
index                                             
1003001363                      832.49  1003001363
1003004201                      136.93  1003004201
1003008095                      686.76  1003008095
1003012428                      272.20  1003012428
1003013608                      155.89  1003013608
...                                ...         ...
1972663144                       79.24  1972663144
1972665321                      752.56  1972665321
1972665594                      188.39  1972665594
1972668416                      382.96  1972668416
1972669489                      259.23  1972669489

[29398 rows x 2 columns]

In [47]:
TEST = Feature_DrugWeighted[0]

In [48]:
#Without groupby
TEST[TEST.NPI == 1003013616]

Empty DataFrame
Columns: [Methocarbamol_Tot_Drug_Cst, NPI]
Index: []

In [49]:
#Chances of Single NPI making double claims for similar medicines
TEST[TEST.NPI == 1003013616]

Empty DataFrame
Columns: [Methocarbamol_Tot_Drug_Cst, NPI]
Index: []

In [50]:
#Merging all columns into final dataframe on NPI

npi_col = FeaturesAll_pd[['NPI']]

w_npi = []

for n, nx in enumerate(Feature_DrugWeighted):
      nggx = pd.merge(npi_col, nx.drop_duplicates(['NPI']), on='NPI', how='left')
      w_npi.append(nggx)

In [51]:
#list w_npi has all columns with values
w_npi[10]

NPI  Risperidone Microspheres_Tot_Drug_Cst
0       1003000126                                    NaN
1       1003000423                                    NaN
2       1003000720                                    NaN
3       1003001785                                    NaN
4       1003001884                                    NaN
...            ...                                    ...
965051  1972641801                                    NaN
965052  1972643476                                    NaN
965053  1972655934                                    NaN
965054  1972662203                                    NaN
965055  1972669075                                    NaN

[965056 rows x 2 columns]

In [52]:
FeaturesAll_pd1 = FeaturesAll_pd

In [53]:
for wx in w_npi:
    
    #Selecting drugname_category
    col_n = wx.columns[1]
    
    #adding new column with values to dataframe
    FeaturesAll_pd1[col_n] = wx[col_n].values
    FeaturesAll_pd1[col_n].fillna(0,inplace=True)

C:\Users\Shailesh\AppData\Local\Temp\ipykernel_13220\2702535886.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  FeaturesAll_pd1[col_n] = wx[col_n].values
C:\Users\Shailesh\AppData\Local\Temp\ipykernel_13220\2702535886.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  FeaturesAll_pd1[col_n] = wx[col_n].values
C:\Users\Shailesh\AppData\Local\Temp\ipykernel_13220\2702535886.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

In [54]:
FeaturesAll_pd1.columns

Index(['NPI', 'State', 'Speciality', 'max_Tot_Drug_Cst', 'sum_Tot_Drug_Cst',
       'avg_Tot_Drug_Cst', 'max_Total_claims', 'sum_Total_claims',
       'avg_Total_claims', 'max_Tot_Day_Suply',
       ...
       'Quetiapine Fumarate_Tot_Day_Suply',
       'Lisinopril/Hydrochlorothiazide_Tot_Clms',
       'Lisinopril/Hydrochlorothiazide_Tot_Day_Suply', 'Diazepam_Tot_Clms',
       'Diazepam_Tot_Day_Suply', 'Pen Needle, Diabetic_Tot_Drug_Cst',
       'Pen Needle, Diabetic_Tot_Day_Suply', 'Tramadol Hcl_Tot_Drug_Cst',
       'Tramadol Hcl_Tot_Clms', 'Tramadol Hcl_Tot_Day_Suply'],
      dtype='object', length=122)

In [55]:
FeaturesAll_pd1.isna().sum()

NPI                                   0
State                                 0
Speciality                            1
max_Tot_Drug_Cst                      0
sum_Tot_Drug_Cst                      0
                                     ..
Pen Needle, Diabetic_Tot_Drug_Cst     0
Pen Needle, Diabetic_Tot_Day_Suply    0
Tramadol Hcl_Tot_Drug_Cst             0
Tramadol Hcl_Tot_Clms                 0
Tramadol Hcl_Tot_Day_Suply            0
Length: 122, dtype: int64

In [56]:
FeaturesAll_pd1.columns

Index(['NPI', 'State', 'Speciality', 'max_Tot_Drug_Cst', 'sum_Tot_Drug_Cst',
       'avg_Tot_Drug_Cst', 'max_Total_claims', 'sum_Total_claims',
       'avg_Total_claims', 'max_Tot_Day_Suply',
       ...
       'Quetiapine Fumarate_Tot_Day_Suply',
       'Lisinopril/Hydrochlorothiazide_Tot_Clms',
       'Lisinopril/Hydrochlorothiazide_Tot_Day_Suply', 'Diazepam_Tot_Clms',
       'Diazepam_Tot_Day_Suply', 'Pen Needle, Diabetic_Tot_Drug_Cst',
       'Pen Needle, Diabetic_Tot_Day_Suply', 'Tramadol Hcl_Tot_Drug_Cst',
       'Tramadol Hcl_Tot_Clms', 'Tramadol Hcl_Tot_Day_Suply'],
      dtype='object', length=122)

In [57]:
FeaturesAll_pd1.to_csv("../data/hyp_pay_fill_0.1 -1.csv")